# Multiprocessing



In [3]:
import sys
import os

sys.path.insert(0, "python")

import multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt
from ishigami import *
from functools import partial

from auxiliary import * 


In [5]:
num_outer, num_inner, which = 100, 100, 1

stat_1 = get_conditional_variance_fast(num_outer, num_inner, which)
stat_2 = get_conditional_variance_readable(num_outer, num_inner, which)      

np.testing.assert_equal(stat_1, stat_2)

In [6]:
num_outer, num_inner = 1000, 1000

%timeit get_conditional_variance_readable(num_outer, num_inner, which)
%timeit get_conditional_variance_fast(num_outer, num_inner, which)

KeyboardInterrupt: 

In [254]:
# Uncertainty propagation
num_draws = 100

%timeit get_unconditional_variance_readable(num_draws)
%timeit get_unconditional_variance_fast(num_draws)

642 ms ± 28.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
897 ms ± 52.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
num_draws = 10000000
var_Y = get_unconditional_variance_fast(num_draws)

7.7909267403998195

In [289]:
num_outer, num_inner, which = 10000, 10000, 2
get_conditional_variance_fast(num_outer, num_inner, which) / var_Y

0.00010243480230571343

In [292]:
get_ishigami_conditional_variances()

(4.345888023894163, 0.06124999999999999, 0.0)

In [296]:
num_outer, num_inner, which = 10000, 10000, 1
get_conditional_variance_fast(num_outer, num_inner, 1) 

0.06214586268043381

We first want to get a sense how many CPU's we have available.

In [4]:
print(f"Number of cpu : {mp.cpu_count()}")

Number of cpu : 4


In [238]:
inputs =  np.random.uniform(low=-np.pi, high=np.pi, size=(num_draws, 3))

def ishigami_readable(x, a=0.7, b=0.1):
    return np.sin(x[0]) + a * np.sin(x[1]) ** 2 + b * x[2] ** 4 * np.sin(x[0])

def ishigami_fast(x, a=0.7, b=0.1):
    x0, x1, x2 = x[:, 0], x[:, 1], x[:, 2]
    return np.sin(x0) + a * np.sin(x1) ** 2 + b * x2 ** 4 * np.sin(x0)

rslt = list()
for input_ in inputs:
    rslt.append(ishigami_readable(input_))

np.testing.assert_equal(ishigami_fast(inputs), rslt)

In [ ]:
# show groups of processes working on total and main effects., mpi worker setup nicely done as well as task differs.

## mp.Process

* without communication

In [276]:
num_outer, num_inner = 1000, 1000

import multiprocessing

def task(num_outer, num_inner, which):
    
    print(f"... started on input parameter {which}")
    rslt = get_conditional_variance_fast(num_outer, num_inner, which)
    print(f"... finished input parameter {which}")

if __name__ == '__main__':
    jobs = []
    for which in range(3):
        p = mp.Process(target=task, args=(num_outer, num_inner, which))
        jobs.append(p)
        p.start()

... started on input parameter 0
... started on input parameter 1
... started on input parameter 2
... finished input parameter 0
... finished input parameter 1
... finished input parameter 2


* with communication

In [275]:
def task(num_outer, num_inner, which, qout):
    rslt = get_conditional_variance_fast(num_outer, num_inner, which)
    qout.put((which, rslt))

if __name__ == '__main__':
    
    processes = []
    qout = mp.Queue()
    
    for which in range(3):
        p = mp.Process(target=task, args=(num_outer, num_inner, which, qout))
        processes.append(p)
        
    for p in processes:
        p.start()

    for p in processes:
        p.join()
        
    unsorted_result = [qout.get() for p in processes]
    result = [t[1] for t in sorted(unsorted_result)] 
    print(result)

... started on input parameter 0
... started on input parameter 1
... started on input parameter 2
... finished input parameter 2
... finished input parameter 0
... finished input parameter 1
[4.327126134649515, 0.07078624789050189, 0.006580924974628393]


## mp.Pool

In [213]:
pool = mp.Pool(processes = 3)
task_partial = partial(task, num_outer, num_inner)
pool.map(task_partial, range(3))

pool.close()
pool.join()

... started on input parameter 1
... started on input parameter 0
... started on input parameter 2
... finished input parameter 2
... finished input parameter 0
... finished input parameter 1


## References

* https://www.journaldev.com/15631/python-multiprocessing-example#python-multiprocessing-process-queue-and-locks

* https://pymotw.com/2/multiprocessing/basics.html